In [4]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

# 加载手写数字数据集（MNIST等）
# 加载手写数字数据集（MNIST）
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# 数据预处理
x_train = x_train.reshape(-1, 28, 28, 1)[:10000].astype('float32') / 255.0
x_test = x_test.reshape(-1, 28, 28, 1)[:1000].astype('float32') / 255.0
y_train = keras.utils.to_categorical(y_train)[:10000]
y_test = keras.utils.to_categorical(y_test)[:1000]


# 定义不同的XXX
if_use_dropout = [0, 1]
# 记录训练结果
train_results = []

for use in if_use_dropout:
    # 定义卷积神经网络模型
    if use == 0:
        model = tf.keras.Sequential([
            # 定义网络结构...
            keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Flatten(),
            keras.layers.Dense(10, activation='softmax')
        ])

    if use == 1:
        model = tf.keras.Sequential([
            # 定义网络结构...
            keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01), input_shape=(28, 28, 1)),
            keras.layers.MaxPooling2D((2, 2)),
            keras.layers.Flatten(),
            keras.layers.Dense(10, activation='softmax')
        ])
        # Dropout
        model.add(tf.keras.layers.Dropout(0.5))
    
    model.optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)

    # 记录梯度范数的变化
    gradient_norms_sgd_m_dp = []

    # 定义训练过程
    num_epochs = 5
    batch_size = 200
    num_batches = len(x_train) // batch_size

    for epoch in range(num_epochs):
        dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(x_train)).batch(batch_size)
        for x_batch, y_batch in dataset:
            # 执行一次前向传播和反向传播，并更新模型参数
            with tf.GradientTape() as tape:
                logits = model(x_batch)
                loss_value = tf.keras.losses.categorical_crossentropy(y_batch, logits)
            
            gradients = tape.gradient(loss_value, model.trainable_variables)
            
            # 更新优化器的参数gradient_norms_adagrad
            tf.keras.optimizers.RMSprop().apply_gradients(zip(gradients, model.trainable_variables))
                
            # 计算梯度范数并记录
            gradient_norm = tf.linalg.global_norm(gradients).numpy()
            gradient_norms_sgd_m_dp.append(gradient_norm)

    # 记录训练结果
    train_results.append((use, gradient_norms_sgd_m_dp))

In [5]:
# 绘制梯度范数变化曲线
plt.figure(figsize=(12,6), dpi = 1080)
plt.plot(train_results[0][1], label = "No Dropout")
plt.plot(train_results[1][1], label = "Use Dropout")
plt.xlabel("Epoch")
plt.ylabel("Gradient Norm")
plt.title("Gradient Norm with Dropout")
plt.legend()
plt.show()